In [ ]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [ ]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from itertools import permutations
from math import sqrt
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps
from wand.image import Image as ImageMagick

In [ ]:
def create_grid(collection, method):
    length = len(collection)
    root = sqrt(length)
    root = int(root)
    
    column = row = root
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    border = 50
    offset = 10
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )
        
    grid = ImageOps.expand(
        grid, 
        border=border, 
        fill=(255)
    )
    
    draw = ImageDraw.Draw(grid)
    font = ImageFont.truetype("FONTS/arial.ttf", 24)
    
    w, h = grid.size

    draw.text(
        (w // 2, 10),
        method,
        (0),
        font=font
    )
    
    return grid

In [ ]:
def create_image(buffer):
    if isinstance(buffer, np.ndarray):
        image = Image.fromarray(~buffer)
    else:
        image = Image.open(buffer)
    
    return image

In [ ]:
def preprocess_image(spectrogram):
    with ImageMagick.from_array(~spectrogram) as image:
        image.despeckle()
        
        image.opaque_paint(
            target='white',
            fill='white',
            fuzz=0.30 * image.quantum_range,
            invert=False
        )

        image.gamma(0.6)

        blob = image.make_blob(format='png')

        buffer = BytesIO(blob)
        return buffer

In [ ]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [ ]:
# minimum = 10
# maximum = minimum + 10

# subset = dataframe.iloc[minimum:maximum]
subset = dataframe.sample(25)

In [ ]:
image = deepcopy(
    subset.spectrogram.to_numpy()
)

image = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(image)

grid = create_grid(image, 'Before')
grid.show()

# grid.save('before.png', format='png')

In [ ]:
image = deepcopy(
    subset.spectrogram.to_numpy()
)
image = np.frompyfunc(
    lambda x: preprocess_image(x),
    1,
    1
)(image)

image = np.frompyfunc(
    lambda x: create_image(x),
    1,
    1
)(image)

grid = create_grid(image, 'After')
grid.show()

# grid.save('after.png', format='png')